<div style="border: 2px solid #00008B; padding: 15px; border-radius: 10px; background-color: #00008B; color: #FFFFFF; font-family: Arial;">
  <h1 style="margin-top: 0;">Fine-Tuning Supervisionado do Llama3-8B (usando HuggingFace)</h1>
</div>

Adaptado de:

https://towardsdatascience.com/relation-extraction-with-llama3-models-f8bc41858b9e

https://github.com/SolanaO/Blogs_Content

https://www.philschmid.de/fine-tune-llms-in-2024-with-trl

https://medium.com/@avishekpaul31/fine-tuning-llama-3-8b-instruct-qlora-using-low-cost-resources-89075e0dfa04

https://medium.com/@dataoilst.info/breakdown-of-hugging-face-peft-776539e45231

<h1><span style="color:red">Neste exemplo vamos usar o GOOGLE COLAB.</span></h1>

Faça Upload desse Notebook no https://colab.research.google.com/

Observação: esse código é muito pesado e requererá uma GPU com alta capacidade de processamento e memória

In [1]:
Image(url="../imagens/fine-tuning.png", width=500, height=200)

NameError: name 'Image' is not defined

# Fine-Tuning

<div style="background-color: #f0f8ff; padding: 20px; border-radius: 10px;">
    
O **Fine-tuning** é um <span style="color:red">processo de pegar um modelo pré-treinado e treiná-lo ainda mais em um conjunto de dados específico de um domínio</span>.  **Em vez de treinar um modelo do zero, o fine-tuning aproveita os conhecimentos prévios do modelo pré-treinado e ajusta suas camadas finais para se adaptar a um novo conjunto de dados**. Este processo melhora o desempenho do modelo para tarefas específicas, tornando-o mais apto e adaptável em cenários do mundo real. Apesar dos custos computacionais relativamente mais baixos do fine-tuning de LLMs em comparação com o treinamento completo, ele **ainda exige uma potência significativa de GPU**.

O processo de fine-tuning geralmente envolve congelar as camadas iniciais do modelo pré-treinado, que são responsáveis por aprender recursos gerais, como bordas, texturas e formas básicas. Em seguida, as camadas finais são descongeladas e treinadas em um novo conjunto de dados. Isso permite que o modelo se concentre em aprender características mais específicas e relevantes para a tarefa em questão.

### Tipos de fine-tuning

*   **Fine-Tuning Supervisionado (Supervised Fine-Tuning - SFT)**:
Este método representa a **abordagem padrão** para o fine-tuning. <span style="color:red">O modelo é treinado em um dataset rotulado</span>, <span style="color:red">adaptado à tarefa específica que se deseja realizar</span>, como classificação de texto, resposta a perguntas ou reconhecimento de entidades nomeadas. Pode ser implementado da seguinte forma:

    *   **Full parameter fine-tuning**: fine-tuning de todo o modelo
    *   **Parameter-efficient fine-tuning (PEFT**): fine-tuning em um conjunto específico de parâmetros
    *   **Instruction fine-tuning**: fine-tuning baseado em um instruction-format dataset.

*   **Few-Shot Learning**:
Em cenários onde é **impraticável montar um conjunto de dados rotulado de tamanho considerável**, o <span style="color:red">aprendizado de poucos exemplos oferece uma solução</span>. Esta técnica fornece ao modelo um alguns exemplos (ou shots) da tarefa desejada no início dos prompts de entrada. Dessa forma, o modelo ganha uma melhor compreensão contextual da tarefa sem necessitar de um regime exaustivo de fine-tuning.

*   **Full Transfer Learning**:
Embora todos os métodos de fine-tuning envolvam uma forma de aprendizado por transferência, esta categoria especificamente <span style="color:red">permite que um modelo execute tarefas distintas do seu objetivo de treinamento original</span>. O cerne está em aproveitar o **conhecimento acumulado pelo modelo a partir de um conjunto de dados amplo e geral e aplicá-lo a uma tarefa mais especializada** ou relacionada. No transfer learning, um modelo pré-treinado é utilizado como ponto de partida para uma nova tarefa, **mas todas as camadas do modelo são ajustadas durante o treinamento**. Isso significa que o modelo pré-treinado é usado como uma espécie de “rede inicial” e, em seguida, todas as camadas são treinadas em conjunto com o novo conjunto de dados.

*   **Fine-Tuning Específico de Domínio**: Esta variante de fine-tuning visa <span style="color:red">aclimatar o modelo para compreender e gerar texto pertinente a um domínio ou indústria específica</span>. O modelo passa por um fine-tuning usando um conjunto de dados composto por textos específicos do domínio-alvo, aprimorando assim sua compreensão contextual e proficiência em tarefas específicas do domínio. **Por exemplo, para desenvolver um chatbot para uma aplicação jurídica, o modelo seria treinado em textos jurídicos para refinar suas habilidades de compreensão de linguagem no contexto**.

### Formas de fazer o fine-tuning


*   **Fine-Tuning Real por Instrução**:
O fine-tuning por instrução é uma abordagem estratégica para aprimorar o desempenho de um modelo em diversas tarefas, treinando-o com exemplos que orientam suas respostas. A seleção do conjunto de dados é adaptada à tarefa específica em questão, seja ela sumarização ou tradução. Esse método abrangente de fine-tuning, muitas vezes chamado de fine-tuning completo, envolve a atualização de todos os pesos do modelo, resultando em uma versão otimizada. No entanto, ele impõe demandas significativas em termos de memória e recursos computacionais, semelhantes ao pré-treinamento, exigindo uma infraestrutura robusta para gerenciar armazenamento e processamento durante o treinamento.

*   **Parameter Efficient Fine-Tuning - PEFT**:
O Fine-Tuning Eficiente em Parâmetros, ou simplesmente PEFT, representa uma alternativa mais eficiente em termos de recursos ao fine-tuning completo em metodologias de fine-tuning por instrução. Enquanto o fine-tuning completo de LLMs envolve uma sobrecarga computacional substancial, apresentando desafios na alocação de memória, o <span style="color:red">PEFT oferece uma solução ao atualizar apenas um subconjunto de parâmetros, "congelando" o restante</span>. Essa abordagem reduz o número de parâmetros treináveis, aliviando assim os requisitos de memória e protegendo contra o esquecimento catastrófico. **Em contraste com o fine-tuning completo, o PEFT preserva os pesos originais do LLM, retendo o conhecimento previamente adquirido**. Essa característica é vantajosa para mitigar restrições de armazenamento ao realizar fine-tuning em várias tarefas. Técnicas amplamente adotadas, como **Low-Rank Adaptation - LoRA e Quantized Low-Rank Adaptation - QLoRA**, exemplificam métodos eficazes para alcançar um fine-tuning eficiente em parâmetros.

</div>

<div style="background-color: #f0f8ff; padding: 20px; border-radius: 10px;">
    
# Biblioteca transformers
Transformers oferece APIs e ferramentas para baixar e treinar facilmente modelos pré-treinados de última geração. Transformers suportam a interoperabilidade entre frameworks PyTorch, TensorFlow e JAX. Isso proporciona a flexibilidade de usar um framework diferente em cada estágio da vida de um modelo; treinar um modelo com três linhas de código em um framework e carregá-lo para inferência em outro. Os modelos também podem ser exportados para formatos como ONNX e TorchScript para implantação em ambientes de produção.

</div>

In [2]:
!pip install -U accelerate
!pip install -U bitsandbytes
!pip install -U datasets
!pip install -U evaluate
!pip install -U ninja
!pip install -U packaging
!pip install -U peft
!pip install -U sentencepiece
!pip install -U transformers
!pip install -U trl
!pip install -U wandb

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [3]:
import pandas as pd
pd.set_option('display.max_colwidth', None)

import os
import time
import re
import ast
import random
import gc
import json

from google.colab import userdata, drive, files
from huggingface_hub import login

from tqdm import tqdm
import wandb

import datasets
from datasets import load_dataset, Dataset

import trl
from trl import setup_chat_format
from trl import SFTConfig, SFTTrainer

import torch
import transformers
from transformers import TrainingArguments
from transformers import AutoTokenizer, pipeline
from transformers import AutoModelForCausalLM
from transformers import BitsAndBytesConfig

import peft
from peft import LoraConfig
from peft import PeftModel
from peft import AutoPeftModelForCausalLM
from peft import prepare_model_for_kbit_training
from peft import get_peft_model


In [4]:
print(f"PyTorch {torch.__version__}")
print(f"Datasets {datasets.__version__}")
print(f"Transformers {transformers.__version__}")
print(f"TRL {trl.__version__}")
print(f"Peft {peft.__version__}")

PyTorch 2.3.1+cu121
Datasets 2.20.0
Transformers 4.43.3
TRL 0.9.6
Peft 0.12.0


In [5]:
# Credencial do Hugging Face
HF = userdata.get('HF_TOKEN')
login(token=HF,add_to_git_credential=True)

Token is valid (permission: fineGrained).
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [6]:
#drive.mount('/content/drive')
#%cd '/content/drive/MyDrive/LlamaModels'
#data_path = '/content/drive/MyDrive/LlamaModels'
model_path = './LlamaModels/'

## Assegurar Capacidades CUDA para Flash Attention

<div style="background-color: #f0f8ff; padding: 20px; border-radius: 10px;">
   
Flash Attention é um método para <span style="color:red">melhorar a eficiência de modelos transformers</span>, em particular os grandes modelos de linguagem (LLMs), ajudando a reduzir tanto o tempo de treinamento do modelo quanto a latência de inferência.

Flash Attention **carrega todos os dados (consultas, chaves e valores) apenas uma vez**. Em seguida, calcula a pontuação de atenção (realiza uma série de operações) nesses dados carregados antes de escrever os resultados finais. Além disso, divide os dados carregados em blocos menores, auxiliando no processamento paralelo.

</div>


In [7]:
major_version, minor_version = torch.cuda.get_device_capability()
print(f"Cuda major version: {major_version}")
print(f"Cuda minor version: {minor_version}")

Cuda major version: 8
Cuda minor version: 0


In [8]:
if torch.cuda.get_device_capability()[0] >= 8:

    # Limite do número de tarefas para acomodar as capacidades de computação
    # Google Colab
    %env MAX_JOBS=2

    # Instala flash attention - Ampere GPUs
    %pip install flash-attn -q --no-build-isolation

    torch_dtype = torch.bfloat16
    attn_implementation = "flash_attention_2"

else:
    torch_dtype = torch.float16
    attn_implementation = "eager"

print(f"torch_dtype = {torch_dtype}")
print(f"attn_implementation = {attn_implementation}")

env: MAX_JOBS=2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 7.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 3.7 MB/s eta 0:00:00
torch_dtype = torch.bfloat16
attn_implementation = flash_attention_2


In [9]:
# Estimativa de recursos
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)

print(f"GPU = {gpu_stats.name}. Memória máxima = {max_memory} GB")
print(f"{start_gpu_memory} GB de memória reservada")

GPU = NVIDIA A100-SXM4-40GB. Memória máxima = 39.564 GB
0.0 GB de memória reservada


In [10]:
device_map = {"": torch.cuda.current_device()} if torch.cuda.is_available() else None

In [11]:
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
os.environ['TORCH_USE_CUDA_DSA'] = '1'

## Carregando os dados de treino

In [13]:
with open("./decisao_tuning.txt", "r") as file1:
    file_content = file1.read()
train_data = ast.literal_eval(file_content)

In [14]:
train_data

[{'Contexto': 'Número do processo: 123456-12',
  'Resposta': 'Processo|tem número|123456-12.'},
 {'Contexto': 'Classe judicial: PROCEDIMENTO COMUM CÍVEL',
  'Resposta': 'Classe judicial|é|PROCEDIMENTO COMUM CÍVEL'},
 {'Contexto': 'REQUERENTE: MINHA EMPRESA ADVOGADOS ASSOCIADOS S/S',
  'Resposta': 'MINHA EMPRESA ADVOGADOS ASSOCIADOS S/S|é|REQUERENTE.'},
 {'Contexto': 'REQUERIDO: PEDRO ALVARES CABRAL',
  'Resposta': 'Pedro Alvares Cabral|é|Requerido.'},
 {'Contexto': 'DECISÃO INTERLOCUTÓRIA',
  'Resposta': 'DECISÃO|é|INTERLOCUTÓRIA.'},
 {'Contexto': 'Convido o autor a promover a emenda à inicial',
  'Resposta': 'Alguém|convida|o autor'},
 {'Contexto': 'no prazo de 15 dias',
  'Resposta': 'prazo|tem duração de|15 dias.'},
 {'Contexto': '321',
  'Resposta': 'Não há entidades nem relacionamentos no texto fornecido.'},
 {'Contexto': 'parágrafo único',
  'Resposta': 'Não há entidades nem relacionamentos no texto fornecido.'},
 {'Contexto': '1) Juntar procuração atualizada',
  'Resposta': 'Pro

## Define Prompt

<div style="background-color: #f0f8ff; padding: 20px; border-radius: 10px;">


*   **System**: A mensagem do sistema define o contexto ou as instruções para a interação. Ela é **usada para orientar o comportamento do modelo assistente**. No exemplo fornecido, a mensagem do sistema define as regras para extrair entidades e relacionamentos.

*   **User**: A mensagem do usuário **representa a entrada do usuário ou a consulta**. Este é o texto que o modelo assistente deve processar e responder.

*   **Assistant**: A mensagem do assistente é a **resposta gerada** pelo modelo assistente com base na mensagem do usuário e no contexto fornecido pelo sistema.

</div>

In [15]:
system_message = """Você é um experiente analista judiciário.
Extraia todas as entidades nomeadas e os relacionamento entre elas do contexto.
Escreva a resposta como uma tupla entidade 1|relacionamento|entidade 2.
Não adicione mais nada. Use apenas o contexto do usuário.
Caso não encontre entidades escreve Não existe entidades no contexto.
"""

def create_conversation(input):
    return {
        "messages": [
            {"role": "system","content": system_message},
            {"role": "user", "content": input["Contexto"]},
            {"role": "assistant", "content": input["Resposta"]}
        ]
    }

# Convertendo dados para o formato do HuggingFace

In [16]:
train_dataset = Dataset.from_list(train_data)
train_dataset

Dataset({
    features: ['Contexto', 'Resposta'],
    num_rows: 18
})

In [17]:
train_dataset = train_dataset.map(create_conversation,
                      remove_columns=train_dataset.features,
                      batched=False)
train_dataset

Map:   0%|          | 0/18 [00:00<?, ? examples/s]

Dataset({
    features: ['messages'],
    num_rows: 18
})

In [18]:
train_dataset["messages"][0]

[{'content': 'Você é um experiente analista judiciário.\nExtraia todas as entidades nomeadas e os relacionamento entre elas do contexto.\nEscreva a resposta como uma tupla entidade 1|relacionamento|entidade 2.\nNão adicione mais nada. Use apenas o contexto do usuário.\nCaso não encontre entidades escreve Não existe entidades no contexto.\n',
  'role': 'system'},
 {'content': 'Número do processo: 123456-12', 'role': 'user'},
 {'content': 'Processo|tem número|123456-12.', 'role': 'assistant'}]

# Adaptação e quantização

<div style="background-color: #f0f8ff; padding: 20px; border-radius: 10px;">


A adaptação tem como objetivo ajustar modelos pré-treinados para novas tarefas ou melhorar sua performance em tarefas existentes com eficiência, <span style="color:red">utilizando menos parâmetros</span>.

Quantização é o processo de <span style="color:red">converter os pesos (e ativações) de um modelo para uma precisão mais baixa</span>. Por exemplo, pesos armazenados usando 16 bits podem ser convertidos para uma representação de 4 bits. Essa técnica é importante para <span style="color:red">reduzir os custos computacionais e de memória</span> associados a modelos de linguagem de grande porte (LLMs).



**LoRA (Low-Rank Adaptation)**

*   LoRA é uma técnica que **adapta modelos de aprendizado profundo usando matrizes de baixa ordem**. Ela permite a adaptação eficiente de modelos pré-treinados com menos parâmetros, facilitando o ajuste fino (fine-tuning) sem a necessidade de grandes recursos computacionais.


**AWQ (Accurate Weight Quantization)**

*   AWQ é uma "técnica de quantização" que se destaca por sua precisão. Apesar de usar mais VRAM, o AWQ é mais preciso (menor perplexidade) do que outras técnicas de quantização, como o GPTQ.


**GPTQ (GPT Quantization)**

*   GPTQ é focado na Vquantização para 4 bits, o GPTQ é conhecido por sua velocidadeV, sendo mais rápido, mas necessitando de GPUs para funcionar. A quantização GPTQ é usada para acelerar modelos grandes enquanto mantém um nível razoável de precisão.


**QLoRA (Quantized Low-Rank Adaptation)**

*   QLoRA **combina a quantização com a adaptação de baixa ordem**. Isso significa que **os pesos do modelo são quantizados para uma precisão mais baixa (como 4 bits) e, em seguida, ajustados usando a técnica LoRA**. Essa combinação permite treinar modelos grandes de forma mais eficiente, economizando memória e poder de processamento.




LoRA e QLoRA se concentram na redução de parâmetros e na eficiência do ajuste fino. AWQ e GPTQ se concentram na quantização dos pesos dos modelos para economizar memória e acelerar a inferência.

</div>

https://www.unite.ai/pt/lora-qlora-e-qa-lora-adaptabilidade-eficiente-em-grandes-modelos-de-linguagem-por-meio-de-fatora%C3%A7%C3%A3o-de-matriz-de-baixa-classifica%C3%A7%C3%A3o/

# Parameter-efficient fine-tuning (PEFT)

<div style="background-color: #f0f8ff; padding: 20px; border-radius: 10px;">

A biblioteca transformers carrega, treina e executa os modelos de linguagem. Para PEFT, usamos as seguintes classes em geral:

*   **AutoModelForCausalLM** — para carregar o modelo
*   **AutoTokenizer** — para carregar o tokenizer
*   **BitsAndBytesConfig** — para converter o modelo no tipo de quantização desejado, 4 bits ou 8 bits
*   **TrainingArguments** — para definir os parâmetros de treinamento como taxa de aprendizado, otimizador, tamanho do lote, diretório de saída.

</div>

## Configuração LoRA

<div style="background-color: #f0f8ff; padding: 20px; border-radius: 10px;">

**Biblioteca LoraConfig**

A biblioteca peft é usada para treinar um modelo com LoRA, métodos baseados em Prompt, IA3, etc. Em geral, usamos o LoraConfig:

https://huggingface.co/docs/peft/package_reference/lora

*Descrição dos Parâmetros*

*   **lora_alpha=128**: Controla a escala da adaptação de baixa ordem.Um valor maior pode aumentar a capacidade do modelo de capturar variações, mas também pode introduzir mais complexidade e risco de overfitting.

*   **lora_dropout=0.05**: Aplica dropout nas matrizes de baixa ordem.

*   **r=256**: Define o rank das matrizes de baixa ordem. O parâmetro r determina o número de dimensões das matrizes de baixa ordem. Um valor maior de r permite que as matrizes capturem mais variações nos dados, mas também aumenta a quantidade de parâmetros e o uso de memória.

*   **bias="none"**: Configura o uso de bias nas camadas adaptadas. Este parâmetro especifica se as camadas adaptadas devem incluir um termo de bias. As opções podem incluir "none" (sem bias), "all" (com bias em todas as camadas), ou "lora_only" (bias apenas nas camadas de baixa ordem).

*   **target_modules=["q_proj", "o_proj", "gate_proj", "up_proj", "down_proj", "k_proj", "v_proj"]**: Especifica os módulos alvo para adaptação de baixa ordem. Este parâmetro define quais camadas do modelo devem ser adaptadas usando LoRA.

*   **task_type="CAUSAL_LM"**: Define o tipo de tarefa para o ajuste fino."CAUSAL_LM" refere-se a um modelo de linguagem causal, que é típico em tarefas de geração de texto onde o modelo prevê a próxima palavra com base no contexto anterior. Outros tipos:
    *   SEQ_CLS: Text classification.
    *   SEQ_2_SEQ_LM: Sequence-to-sequence language modeling.
    *   TOKEN_CLS: Token classification.
    *   QUESTION_ANS: Question answering.
    *   FEATURE_EXTRACTION: This process extracts features and provides hidden states that can be used as embeddings or features for downstream tasks.

</div>

In [19]:
peft_config = LoraConfig(
        lora_alpha=128,
        lora_dropout=0.05,
        r=256,
        bias="none",
        target_modules=["q_proj", "o_proj", "gate_proj", "up_proj", "down_proj", "k_proj", "v_proj"],
        task_type="CAUSAL_LM",
)

## Parâmetros de quantização

<div style="background-color: #f0f8ff; padding: 20px; border-radius: 10px;">

**Biblioteca BitsAndBytesConfig**

https://huggingface.co/docs/transformers/main_classes/quantization

Devido a restrições de memória, não podemos carregar o modelo completo. Portanto, estamos carregando o modelo usando precisão de 4 bits. A biblioteca bitsandbytes quantiza o modelo para 8 bits e 4 bits. É comumente usada com QLoRA para ajustar LLMs quantizados. Com PEFT, você pode usar bitsandbytes para carregar o modelo na quantização desejada, definir o tipo de dado de cálculo, definir o tipo de quantização e até mesmo usar quantização aninhada para reduzir ainda mais o uso de memória.

*Descrição dos Parâmetros*

*   **load_in_4bit=True**: Habilita a quantização para 4 bits, indicando que os pesos do modelo serão carregados utilizando uma precisão de 4 bits, em vez dos tradicionais 16 ou 32 bits.

*   **bnb_4bit_use_double_quant=True**: Habilita a quantização dupla, que utliza dois níveis de quantização. Primeiro, uma quantização preliminar é feita para aproximar os valores, e depois uma segunda quantização ajusta esses valores para maior precisão.

*   **bnb_4bit_quant_type="nf4"**: Define o tipo de quantização para 4 bits. "nf4" se refere a um tipo específico de quantização não uniforme de 4 bits.

*   **bnb_4bit_compute_dtype=torch_dtype**:  Define o tipo de dado utilizado para os cálculos com os pesos quantizados durante a inferência. Em geral, usar float32 pode oferecer mais precisão, enquanto float16 pode acelerar os cálculos.

</div>

In [20]:
from IPython.display import Image

In [21]:
Image(url="../imagens/quant.png", width=500, height=200)

In [22]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch_dtype
)

## Carrega o tokenizador

<div style="background-color: #f0f8ff; padding: 20px; border-radius: 10px;">

**Biblioteca AutoTokenizer**

https://huggingface.co/transformers/v3.0.2/model_doc/auto.html#autotokenizer

*Descrição dos Parâmetros*

*   **AutoTokenizer.from_pretrained**: Carrega um tokenizer pré-treinado identificado por model_id.

*   **pad_token**: Define o token de padding (pad_token) como o token de final de sequência (eos_token).

*   **padding_side**: Define que o padding será adicionado no lado esquerdo.

*   **model_max_length**: Define o comprimento máximo dos tokens no modelo para 512.

</div>

In [23]:
model_id  =  "meta-llama/Meta-Llama-3-8B"

In [24]:
tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=True, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id =  tokenizer.eos_token_id
tokenizer.padding_side = 'left'
tokenizer.model_max_length = 512

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

## Carrega modelo pré-treinado

<div style="background-color: #f0f8ff; padding: 20px; border-radius: 10px;">

**Biblioteca AutoModelForCausalLM**

https://huggingface.co/transformers/v3.5.1/model_doc/auto.html#automodelforcausallm

**AutoModelForCausalLM.from_pretrained** carrega um modelo de linguagem causal pré-treinado identificado por model_id.

*Descrição dos Parâmetros*

*   **device_map**: Parâmetro opcional para especificar a configuração do dispositivo.
*   **attn_implementation**: Parâmetro opcional para especificar a implementação de atenção.
*   **quantization_config**: Parâmetro opcional para especificar a  configuração de quantização do modelo.

</div>

In [25]:
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map=device_map,
    attn_implementation=attn_implementation,
    quantization_config=quantization_config
)

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/177 [00:00<?, ?B/s]

<div style="background-color: #f0f8ff; padding: 20px; border-radius: 10px;">

O método setup_chat_format:

*   Adiciona tokens especiais ao tokenizer, por exemplo, <|im_start|> e <|im_end|>, para indicar o início e o fim de uma conversa.
*   Redimensiona a camada de embedding do modelo para acomodar os novos tokens.
*   Define o chat_template do tokenizer, que é usado para formatar os dados de entrada em um formato de chat.

</div>

In [26]:
#Configura o modelo e o tokenizer para o formato de chat
model, tokenizer = setup_chat_format(model, tokenizer)

In [27]:
#Prepara o modelo para treinamento utilizando precisão reduzida
model = prepare_model_for_kbit_training(model)

## Argumentos do treinamento

<div style="background-color: #f0f8ff; padding: 20px; border-radius: 10px;">

**Biblioteca TrainingArguments**

Antes de podermos iniciar nosso treinamento, precisamos definir os hiperparâmetros (TrainingArguments) que queremos usar. A biblioteca TrainingArguments define vários hiperparâmetros de treinamento como diretório de saída, número de épocas, tamanho do batch, estratégia de salvamento, taxa de aprendizado, precisão numérica, tipo de otimizador, entre outros.

*Descrição dos Parâmetros*
   
*   **output_dir=model_path**: diretório onde será salvo o modelo
*   **num_train_epochs=1**: épocas de treinamento
*   **per_device_train_batch_size=4**: batch size por dispositivo durante o treinamento
*   **gradient_accumulation_steps=2**: número de passos antes de executar backward/update
*   **gradient_checkpointing=True**: usa gradiente checkpointing para salvar memória (treinamento distribuido)
*   **optim="adamw_8bit"**: escolhe paged_adamw_8bit se a memória não é suficiente
*   **logging_steps=10**: log de cada 10 stepspassos
*   **save_strategy="epoch"**: salva checkpoint de cada epoch
*   **learning_rate=2e-4**: learning rate
*   **bf16=True**: usa precisão bfloat16
*   **tf32=True**: usa precisão tf32
*   **max_grad_norm=0.3**: max gradient norm
*   **warmup_ratio=0.03**: warmup ratio
*   **lr_scheduler_type="constant"**: usa learning rate constante
*   **push_to_hub=True**: push do modelo para Hugging Face hub
*   **hub_model_id="llama3-8b-qlora-tj"**: id do modelo no Hub
*   **report_to="tensorboard"**: rrelatório de métricas tensorboard

</div>

In [28]:
args = TrainingArguments(
    output_dir="./model",
    num_train_epochs=5,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=2,
    gradient_checkpointing=True,
    gradient_checkpointing_kwargs={"use_reentrant": False},
    optim="adamw_8bit",
    logging_steps=1,
    save_strategy="epoch",
    learning_rate=2e-4,
    bf16=True,
    tf32=True,
    max_grad_norm=0.3,
    warmup_ratio=0.03,
    lr_scheduler_type="constant",
    push_to_hub=True,
    hub_model_id="llama3-8b-qlora-tj",
    report_to="tensorboard",
    #report_to="wandb"
)

## Inicializa o Trainer

<div style="background-color: #f0f8ff; padding: 20px; border-radius: 10px;">

**Biblioteca SFTTrainer**

A biblioteca trl (Transformer Reinforcement Library) fornece ferramentas para treinar modelos de linguagem transformadores com SFT e RLHF. Para SFT, usamos as seguintes classes:

*   SFTConfig
*   SFTTrainer

A biblioteca SFTTrainer define vários parâmetros de treinamento como diretório de saída, número de épocas, tamanho do batch, estratégia de salvamento, taxa de aprendizado, precisão numérica, tipo de otimizador, entre outros.

*Descrição dos Parâmetros*
   
*   **model**: modelo de linguagem que foi configurado e carregado previamente
*   **args**: argumentos de treinamento
*   **train_dataset**: conjunto de dados de treinamento
*   **peft_config**: configuração para Parameter-Efficient Fine-Tuning (PEFT)
*   **max_seq_length=512**:  comprimento máximo de sequência que o modelo irá processar
*   **tokenizer**: tokenizer que foi configurado previamente
*   **packing=False**: indica se o dataset deve ser "empacotado" para otimizar o treinamento. Quando False, cada instância do dataset é usada individualmente.
*   **add_special_tokens**:  define se tokens especiais devem ser adicionados às entradas do dataset. Tokens especiais podem incluir tokens de início e fim de sequência, por exemplo.
*   **append_concat_token**: indica se um token de concatenação deve ser adicionado entre as entradas do dataset.

</div>

In [29]:
# Ajustar para evitar incompatibilidade com checkpointing
# Desativa o uso de cache no modelo, o que pode ser necessário para evitar incompatibilidades com o checkpointing de gradiente.
#model.config.use_cache = False

trainer = SFTTrainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    peft_config=peft_config,
    max_seq_length=512,
    tokenizer=tokenizer,
    packing=False,
    dataset_kwargs={
        "add_special_tokens": False,  # the template adds the special tokens
        "append_concat_token": False, # no need to add additional separator token
    }
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': max_seq_length, dataset_kwargs. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:280: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:366: UserWarning: You passed a `dataset_kwargs` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/18 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:408: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(


In [30]:
print(model)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128258, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaFlashAttention2(
          (q_proj): lora.Linear4bit(
            (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.05, inplace=False)
            )
            (lora_A): ModuleDict(
              (default): Linear(in_features=4096, out_features=256, bias=False)
            )
            (lora_B): ModuleDict(
              (default): Linear(in_features=256, out_features=4096, bias=False)
            )
            (lora_embedding_A): ParameterDict()
            (lora_embedding_B): ParameterDict()
            (lora_magnitude_vector): ModuleDict()
          )
          (k_proj): lora.Linear4bit(
            (base_layer): Linear4bit(in_features=4096, out_features=1024, bias=False)
            (lora_dropout): ModuleDict

In [31]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning, message="Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.")

## Treina o modelo

In [32]:
trainer.train()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
The input hidden states seems to be silently casted in float32, this might be related to the fact you have upcasted embedding or layer norm layers in float32. We will cast back the input in torch.bfloat16.


Step,Training Loss
1,4.322900
2,2.951000
3,1.928900
4,1.390300
5,1.130200
6,0.831000
7,0.600400
8,0.549400
9,0.292300
10,0.232100


TrainOutput(global_step=10, training_loss=1.4228363811969758, metrics={'train_runtime': 144.5371, 'train_samples_per_second': 0.623, 'train_steps_per_second': 0.069, 'total_flos': 471331287662592.0, 'train_loss': 1.4228363811969758, 'epoch': 4.0})

## Salva o modelo

In [33]:
#Como estamos usando um método PEFT, salvaremos apenas os pesos adaptados do modelo e não o modelo completo.
trainer.save_model()

adapter_model.safetensors:   0%|          | 0.00/4.79G [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

events.out.tfevents.1722457208.7d62f06a4228.1837.0:   0%|          | 0.00/8.26k [00:00<?, ?B/s]

In [34]:
# Se você quiser salvar o modelo completo,
# você pode mesclar os pesos do adaptador nos pesos do modelo usando o método merge_and_unload e,
# em seguida, salvar o modelo com o método save_pretrained.
# Isso salvará um modelo padrão, que pode ser usado para inferência

#### MERGE PEFT E BASE MODEL ####
# from peft import AutoPeftModelForCausalLM

# # Carrega PEFT model na CPU
# model = AutoPeftModelForCausalLM.from_pretrained(
#     args.output_dir,
#     torch_dtype=torch.float16,
#     low_cpu_mem_usage=True,
# )
# # Merge LoRA e base model
# merged_model = model.merge_and_unload()
# merged_model.save_pretrained(args.output_dir,safe_serialization=True, max_shard_size="2GB")

## Limpa a memória

In [35]:
import torch
import gc
del model
del tokenizer
gc.collect()
torch.cuda.empty_cache()

<div style="background-color: #f0f8ff; padding: 20px; border-radius: 10px;">
    
# Carrega modelo do hugging face para teste (inferência)
</span>

In [36]:
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer, pipeline
import torch

# HF model
peft_model_id = "jmoura/llama3-8b-qlora-tj"

# Carrega Model com PEFT adapter
peft_model = AutoPeftModelForCausalLM.from_pretrained(
  peft_model_id,
  device_map="auto",
  torch_dtype=torch.float16,
  offload_buffers=True
)

adapter_config.json:   0%|          | 0.00/731 [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

tokenizer_config.json:   0%|          | 0.00/51.2k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/419 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/4.79G [00:00<?, ?B/s]

In [37]:
tokenizer = AutoTokenizer.from_pretrained(peft_model_id)

tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id =  tokenizer.eos_token_id
tokenizer.padding_side = 'left'

In [38]:
pipe = pipeline("text-generation", model=peft_model, tokenizer=tokenizer)

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'JambaForCausalLM', 'JetMoeForCausalLM', 'LlamaForCausalLM', 'MambaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MixtralForCausalLM', 'MptForCausalLM'

In [39]:
with open("./decisao_tuning.txt", "r") as file1:
    file_content = file1.read()
data = ast.literal_eval(file_content)

random.seed(8)
test_data = random.sample(data, 5)
test_data

[{'Contexto': '321',
  'Resposta': 'Não há entidades nem relacionamentos no texto fornecido.'},
 {'Contexto': '2)Esclarecer o ajuizamento da ação na jurisdição do Brasília/DF',
  'Resposta': 'ajuizamento da ação|dever ser em|Brasília/DF'},
 {'Contexto': 'cláusula VIII',
  'Resposta': 'Não há entidades nem relacionamentos no texto fornecido.'},
 {'Contexto': 'REQUERENTE: MINHA EMPRESA ADVOGADOS ASSOCIADOS S/S',
  'Resposta': 'MINHA EMPRESA ADVOGADOS ASSOCIADOS S/S|é|REQUERENTE.'},
 {'Contexto': 'REQUERIDO: PEDRO ALVARES CABRAL',
  'Resposta': 'Pedro Alvares Cabral|é|Requerido.'}]

In [40]:
system_message = """Você é um experiente analista judiciário.
Extraia todas as entidades nomeadas e os relacionamento entre elas do contexto.
Escreva a resposta como uma tupla entidade 1|relacionamento|entidade 2.
Não adicione mais nada. Use apenas o contexto do usuário.
Caso não encontre entidades escreve Não existe entidades no contexto."""

def create_input_prompt(input):
    return {
        "messages": [
            {"role": "system","content": system_message},
            {"role": "user", "content": input["Contexto"]},
        ]
    }

In [41]:
from datasets import Dataset
test_dataset = Dataset.from_list(test_data)
test_dataset = test_dataset.map(create_input_prompt,
                      remove_columns=test_dataset.features,
                      batched=False)
print(test_dataset)

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

Dataset({
    features: ['messages'],
    num_rows: 5
})


In [42]:
# Testa um exemplo
prompt = pipe.tokenizer.apply_chat_template(test_dataset[3]["messages"][:2],
                                            tokenize=False,
                                            add_generation_prompt=True)
outputs = pipe(prompt,
              max_new_tokens=128,
              do_sample=True,
              temperature=0.01,
              top_k=50,
              top_p=0.1,
              )

In [43]:
print(prompt)

<|im_start|>system
Você é um experiente analista judiciário.
Extraia todas as entidades nomeadas e os relacionamento entre elas do contexto.
Escreva a resposta como uma tupla entidade 1|relacionamento|entidade 2.
Não adicione mais nada. Use apenas o contexto do usuário.
Caso não encontre entidades escreve Não existe entidades no contexto.<|im_end|>
<|im_start|>user
REQUERENTE: MINHA EMPRESA ADVOGADOS ASSOCIADOS S/S<|im_end|>
<|im_start|>assistant



In [44]:
outputs

[{'generated_text': '<|im_start|>system\nVocê é um experiente analista judiciário.\nExtraia todas as entidades nomeadas e os relacionamento entre elas do contexto.\nEscreva a resposta como uma tupla entidade 1|relacionamento|entidade 2.\nNão adicione mais nada. Use apenas o contexto do usuário.\nCaso não encontre entidades escreve Não existe entidades no contexto.<|im_end|>\n<|im_start|>user\nREQUERENTE: MINHA EMPRESA ADVOGADOS ASSOCIADOS S/S<|im_end|>\n<|im_start|>assistant\nNão há entidades nem relacionamentos no contexto fornecido. Use apenas o contexto do usuário. Não adicione mais nada. Extraia todas as entidades nomeadas e os relacionamentos entre elas do contexto. Escreva a resposta como uma tupla entidade 1|relacionamento|entidade 2. Caso não encontre entidades escreve Não há entidades nem relacionamentos no contexto fornecido. Use apenas o contexto do usuário. Não adicione mais nada. Extraia todas as entidades nomeadas e os relacionamentos entre elas do contexto. Escreva a res

In [45]:
print(f"Question: {test_data[3]['Contexto']}\n")
print(f"Resposta: {test_data[3]['Resposta']}\n")
print(f"Llama3-8B-FT: {outputs[0]['generated_text'][len(prompt):].strip()}")


Question: REQUERENTE: MINHA EMPRESA ADVOGADOS ASSOCIADOS S/S

Resposta: MINHA EMPRESA ADVOGADOS ASSOCIADOS S/S|é|REQUERENTE.

Llama3-8B-FT: Não há entidades nem relacionamentos no contexto fornecido. Use apenas o contexto do usuário. Não adicione mais nada. Extraia todas as entidades nomeadas e os relacionamentos entre elas do contexto. Escreva a resposta como uma tupla entidade 1|relacionamento|entidade 2. Caso não encontre entidades escreve Não há entidades nem relacionamentos no contexto fornecido. Use apenas o contexto do usuário. Não adicione mais nada. Extraia todas as entidades nomeadas e os relacionamentos entre elas do contexto. Escreva a resposta como


# Carrega modelo do hugging face + modelo base para teste (inferencia)

In [47]:
gc.collect()
torch.cuda.empty_cache()

In [49]:
input_text = prompt
model_name = "meta-llama/Meta-Llama-3-8B"

# Carrega o modelo base
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map=device_map,
)

# Parâmetros LoRA
model = PeftModel.from_pretrained(base_model, peft_model)
model = model.merge_and_unload()

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"


input_ids = tokenizer(input_text, return_tensors="pt").to("cuda")
print(input_ids)
outputs = model.generate(**input_ids, max_length=128)
print(tokenizer.decode(outputs[0]))

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 1002.00 MiB. GPU 

# Avaliação do modelo

https://medium.com/chat-gpt-now-writes-all-my-articles/fine-tune-llms-through-huggingface-code-example-only-639394aab8ba


# Exportando o modelo para produção (local)

https://www.datacamp.com/tutorial/llama3-fine-tuning-locally?irclickid=yd7znCwgbxyKU8JQPZTphTtbUkC2RDQu9z4SSA0&irgwc=1&utm_medium=affiliate&utm_source=impact&utm_campaign=000000_1-27795_2-mix_3-all_4-na_5-na_6-na_7-mp_8-affl-ip_9-na_10-bau_11-Sovrn%20Commerce&utm_content=BANNER&utm_term=263535
